In [9]:
import pandas as pd 
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import numpy as np

In [38]:

api_key = XXXXXXXXx

# 117th Congress Laws

In [91]:
laws_117 = pd.read_csv("../data/117_laws.csv")
laws_117.head()

,Legislation Number,URL,Congress,Title,Sponsor,Party of Sponsor,Date of Introduction,Latest Action,Latest Action Date,Latest Summary,Amendment Text (Latest)
0,H.R. 9308,https://www.congress.gov/bill/117th-congress/h...,117th Congress (2021-2022),To designate the facility of the United States...,"Jacobs, Sara [Rep.-D-CA-53]",Democratic,11/16/2022,Became Public Law No: 117-314.,12/27/2022,<p>This act designates the facility of the Uni...,NaN
1,H.R. 8982,https://www.congress.gov/bill/117th-congress/h...,117th Congress (2021-2022),Bulk Infant Formula to Retail Shelves Act,"DelBene, Suzan K. [Rep.-D-WA-1]",Democratic,09/26/2022,Became Public Law No: 117-192.,10/10/2022,<p><strong>Bulk Infant Formula to Retail Shelv...,NaN
2,H.R. 8656,https://www.congress.gov/bill/117th-congress/h...,117th Congress (2021-2022),To designate the clinic of the Department of V...,"Banks, Jim [Rep.-R-IN-3]",Republican,08/05/2022,Became Public Law No: 117-179.,09/30/2022,<p>This act designates the clinic of the Depa...,NaN
3,H.R. 8454,https://www.congress.gov/bill/117th-congress/h...,117th Congress (2021-2022),Medical Marijuana and Cannabidiol Research Exp...,"Blumenauer, Earl [Rep.-D-OR-3]",Democratic,07/21/2022,Became Public Law No: 117-215.,12/02/2022,<p><strong>Medical Marijuana and Cannabidiol R...,NaN
4,H.R. 8404,https://www.congress.gov/bill/117th-congress/h...,117th Congress (2021-2022),Respect for Marriage Act,"Nadler, Jerrold [Rep.-D-NY-10]",Democratic,07/18/2022,Became Public Law No: 117-228.,12/13/2022,<p><b>Respect for Marriage Act</b></p> <p>This...,NaN


In [14]:
# Adding columns to make it easier to query API

# getting the chamber of origin
def get_chamber(x):
    if x.startswith("H.R."):
        return "hr"
    elif x.startswith("H.J.Res"):
        return "hjres"
    elif x.startswith("S.J.Res"):
        return "sjres"
    else:
        return "s"

laws_117["Chamber"] = laws_117["Legislation Number"].apply(get_chamber)


In [99]:
laws_117["Chamber"].value_counts()

Chamber
hr       198
s        160
sjres      4
hjres      3
Name: count, dtype: int64

In [17]:
# getting the bill number
def get_bill_number(x):
    return x.split(" ")[1]

laws_117["Bill Number"] = laws_117["Legislation Number"].apply(get_bill_number)
laws_117.head()

In [142]:
laws_117.columns

Index(['Legislation Number', 'URL', 'Congress', 'Title', 'Sponsor',
       'Party of Sponsor', 'Date of Introduction', 'Latest Action',
       'Latest Action Date', 'Latest Summary',
       ...
       'This Act may be cited as the “Artistic Recognition for Talented Students Act” or the “ARTS Act”. Section 708 of title 17, United States Code, is amended by adding at the end the following:“(e)(1) Definition.In this subsection, the term ‘covered competition’ means—“(A) an art competition sponsored by the Congressional Institute that is open only to high school students; or“(B) the competition established under section 3 of House Resolution 77, 113th Congress, agreed to February 26, 2013.“(2) With respect to a work that wins a covered competition, the Register of Copyrights—“(A) Deadline.shall waive the requirement under subsection (a)(1) with respect to an application for registration of a copyright claim for that work if that application is submitted to the Copyright Office not later tha

In [131]:
# get text via api
api_key = XXXXX
congress = 117
df = pd.DataFrame(columns = ['Legislation Number', 'URL', 'Congress', 'Title', 'Sponsor',
       'Date of Introduction', "Chamber", "Bill Number", "Text"])

def get_bill_text(congress, chamber, bill_no, api_key):
    url = f"https://api.congress.gov/v3/bill/{congress}/{chamber}/{bill_no}/text?api_key={api_key}"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200:
        text_url = data['textVersions'][-1]['formats'][2]['url'] 
        get_text = requests.get(text_url)
        soup = BeautifulSoup(get_text.content, 'html.parser')
        contents = soup.find_all('content')
        text_list = [content.text for content in contents]
        text = " ".join(text_list)
        text = text.replace('\u2001', '')
        return text
    else:
        print(response.status_code)
        #return f"Error occurred while making the API request for {congress}th congress and bill number {bill_no}."

In [126]:
data['textVersions'] 

[{'date': None,
  'formats': [{'type': 'Formatted Text',
    'url': 'https://www.congress.gov/112/bills/hr6671/BILLS-112hr6671enr.htm'},
   {'type': 'PDF',
    'url': 'https://www.congress.gov/112/bills/hr6671/BILLS-112hr6671enr.pdf'},
   {'type': 'Formatted XML',
    'url': 'https://www.congress.gov/112/bills/hr6671/BILLS-112hr6671enr.xml'}],
  'type': 'Enrolled Bill'},
 {'date': '2012-12-19T05:00:00Z',
  'formats': [{'type': 'Formatted Text',
    'url': 'https://www.congress.gov/112/bills/hr6671/BILLS-112hr6671rds.htm'},
   {'type': 'PDF',
    'url': 'https://www.congress.gov/112/bills/hr6671/BILLS-112hr6671rds.pdf'},
   {'type': 'Formatted XML',
    'url': 'https://www.congress.gov/112/bills/hr6671/BILLS-112hr6671rds.xml'}],
  'type': 'Received in Senate'},
 {'date': '2012-12-18T05:00:00Z',
  'formats': [{'type': 'Formatted Text',
    'url': 'https://www.congress.gov/112/bills/hr6671/BILLS-112hr6671eh.htm'},
   {'type': 'PDF',
    'url': 'https://www.congress.gov/112/bills/hr6671/BI

In [130]:
data['textVersions'] 
last_entry = data['textVersions'][-1]
if 'url' in last_entry and last_entry['url'].endswith('.xml'):
    text_url = last_entry['url']
text_url

In [ ]:
for i in data['textVersions'][-{i}]['formats']
    for format in formats:
        if format['type'] == 'Formatted XML':
            text_url = format['url']
            break
        else:
            formats = data['textVersions'][-2]['formats']
            for format in formats:
                if format['type'] == 'Formatted XML':
                    text_url = format['url']
                    break

In [47]:
congress = 112
chamber = "hr"
bill_no = 6671
api_key = "jDJYSv2qFcQrFUq7kQEZB3zo019P2QBrfotk4bs1"

url = f"https://api.congress.gov/v3/bill/{congress}/{chamber}/{bill_no}/text?api_key={api_key}"
response = requests.get(url)
data = response.json()
text_url = data['textVersions']
text_url[-1]

[{'date': None,
  'formats': [{'type': 'Formatted Text',
    'url': 'https://www.congress.gov/112/bills/hr6671/BILLS-112hr6671enr.htm'},
   {'type': 'PDF',
    'url': 'https://www.congress.gov/112/bills/hr6671/BILLS-112hr6671enr.pdf'},
   {'type': 'Formatted XML',
    'url': 'https://www.congress.gov/112/bills/hr6671/BILLS-112hr6671enr.xml'}],
  'type': 'Enrolled Bill'},
 {'date': '2012-12-19T05:00:00Z',
  'formats': [{'type': 'Formatted Text',
    'url': 'https://www.congress.gov/112/bills/hr6671/BILLS-112hr6671rds.htm'},
   {'type': 'PDF',
    'url': 'https://www.congress.gov/112/bills/hr6671/BILLS-112hr6671rds.pdf'},
   {'type': 'Formatted XML',
    'url': 'https://www.congress.gov/112/bills/hr6671/BILLS-112hr6671rds.xml'}],
  'type': 'Received in Senate'},
 {'date': '2012-12-18T05:00:00Z',
  'formats': [{'type': 'Formatted Text',
    'url': 'https://www.congress.gov/112/bills/hr6671/BILLS-112hr6671eh.htm'},
   {'type': 'PDF',
    'url': 'https://www.congress.gov/112/bills/hr6671/BI

In [73]:
data['textVersions'][-1]['formats']

[{'type': 'Formatted Text',
  'url': 'https://www.congress.gov/112/plaws/publ258/PLAW-112publ258.htm'},
 {'type': 'PDF',
  'url': 'https://www.congress.gov/112/plaws/publ258/PLAW-112publ258.pdf'}]

In [78]:
formats = data['textVersions'][-1]['formats']
text_url = None

for format in formats:
    if format['type'] == 'Formatted XML':
        text_url = format['url']
        break
if text_url is None:
    formats = data['textVersions'][-2]['formats']
    for format in formats:
        if format['type'] == 'Formatted XML':
            text_url = format['url']
            break
print(text_url)


https://www.congress.gov/112/bills/hr6671/BILLS-112hr6671ih.xml


In [165]:
# run the function on laws_117
for index, row in laws_117.iterrows():
    congress = 117
    chamber = row["Chamber"]
    bill_no = row["Bill Number"]
    legislation_number = row["Legislation Number"]
    url = row["URL"]
    title = row["Title"]
    sponsor = row["Sponsor"]
    date_intro = row["Date of Introduction"]
    text = get_bill_text(congress, chamber, bill_no, api_key)
    new_row = {'Legislation Number': legislation_number, 
                    'URL': url, 
                    'Congress': congress, 
                    'Title': title, 
                    'Sponsor': sponsor,
                    'Date of Introduction': date_intro,
                    "Chamber": chamber,
                    "Bill Number": bill_no,
                    "Text": text}
    df.loc[len(df)] = new_row


/Users/natehiatt/miniconda3/envs/capstone/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
/Users/natehiatt/miniconda3/envs/capstone/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keywo

In [167]:
df.to_csv("../data/117_laws_text.csv", index = False)

In [175]:
# import data
laws_116 = pd.read_csv("../data/116_laws.csv")
df_116 = pd.DataFrame(columns = ['Legislation Number', 'URL', 'Congress', 'Title', 'Sponsor',
        'Date of Introduction', "Chamber", "Bill Number", "Text"])

# create columns
laws_116["Chamber"] = laws_116["Legislation Number"].apply(get_chamber)
laws_116["Bill Number"] = laws_116["Legislation Number"].apply(get_bill_number)

# run function to get text
for index, row in laws_116.iterrows():
    congress = 116
    chamber = row["Chamber"]
    bill_no = row["Bill Number"]
    legislation_number = row["Legislation Number"]
    url = row["URL"]
    title = row["Title"]
    sponsor = row["Sponsor"]
    date_intro = row["Date of Introduction"]
    text = get_bill_text(congress, chamber, bill_no, api_key)
    new_row = {'Legislation Number': legislation_number, 
                    'URL': url, 
                    'Congress': congress, 
                    'Title': title, 
                    'Sponsor': sponsor,
                    'Date of Introduction': date_intro,
                    "Chamber": chamber,
                    "Bill Number": bill_no,
                    "Text": text}
    df_116.loc[len(df_116)] = new_row

/Users/natehiatt/miniconda3/envs/capstone/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
/Users/natehiatt/miniconda3/envs/capstone/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keywo

In [182]:
df_116["Text"].tail(20)

324    This Act may be cited as the “Native American ...
325    This Act may be cited as the “Esther Martinez ...
326    Robert J. Dole, also known as Bob Dole, was bo...
327    This Act may be cited as the “Christa McAuliff...
328                                                     
329                                                     
330    This Act may be cited as the “Indian Community...
331    This Act may be cited as the “Practical Reform...
332                                                     
333    This Act may be cited as the “Alaska Remote Ge...
334    This Act may be cited as the “Supporting Veter...
335    This Act may be cited as the “Pallone-Thune Te...
336    This Act may be cited as the “Combat Online Pr...
337    This Act may be cited as the “Columbia River I...
338                                                     
339    This Act may be cited as the “John D. Dingell,...
340    This Act may be cited as the “Government Emplo...
341     That, in accordance wit

In [183]:
df_116.to_csv("../data/116_laws_text.csv", index = False)

## 115th Congress

In [184]:
# import data
laws_115 = pd.read_csv("../data/115_laws.csv")
df_115 = pd.DataFrame(columns = ['Legislation Number', 'URL', 'Congress', 'Title', 'Sponsor',
        'Date of Introduction', "Chamber", "Bill Number", "Text"])

# create columns
laws_115["Chamber"] = laws_115["Legislation Number"].apply(get_chamber)
laws_115["Bill Number"] = laws_115["Legislation Number"].apply(get_bill_number)

# run function to get text
for index, row in laws_115.iterrows():
    congress = 115
    chamber = row["Chamber"]
    bill_no = row["Bill Number"]
    legislation_number = row["Legislation Number"]
    url = row["URL"]
    title = row["Title"]
    sponsor = row["Sponsor"]
    date_intro = row["Date of Introduction"]
    text = get_bill_text(congress, chamber, bill_no, api_key)
    new_row = {'Legislation Number': legislation_number, 
                    'URL': url, 
                    'Congress': congress, 
                    'Title': title, 
                    'Sponsor': sponsor,
                    'Date of Introduction': date_intro,
                    "Chamber": chamber,
                    "Bill Number": bill_no,
                    "Text": text}
    df_115.loc[len(df_115)] = new_row

/Users/natehiatt/miniconda3/envs/capstone/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
/Users/natehiatt/miniconda3/envs/capstone/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse thi

In [187]:
df_115.head()

,Legislation Number,URL,Congress,Title,Sponsor,Date of Introduction,Chamber,Bill Number,Text
0,H.R. 7327,https://www.congress.gov/bill/115th-congress/h...,115,Strengthening and Enhancing Cyber-capabilities...,"Hurd, Will [Rep.-R-TX-23]",12/19/2018,hr,7327,This Act may be cited as the “Strengthening an...
1,H.R. 7319,https://www.congress.gov/bill/115th-congress/h...,115,To amend the Federal Assets Sale and Transfer ...,"Barletta, Lou [Rep.-R-PA-11]",12/17/2018,hr,7319,
2,H.R. 7318,https://www.congress.gov/bill/115th-congress/h...,115,To amend the Federal Assets Sale and Transfer ...,"Barletta, Lou [Rep.-R-PA-11]",12/17/2018,hr,7318,by striking subparagraph (B); by striking the ...
3,H.R. 7279,https://www.congress.gov/bill/115th-congress/h...,115,Water Infrastructure Improvement Act,"Gibbs, Bob [Rep.-R-OH-7]",12/12/2018,hr,7279,This Act may be cited as the “Water Infrastruc...
4,H.R. 7243,https://www.congress.gov/bill/115th-congress/h...,115,To amend Public Law 115-217 to change the addr...,"Murphy, Stephanie N. [Rep.-D-FL-7]",12/10/2018,hr,7243,"In section 1(a) of Public Law 115–217,Ante, p...."


In [188]:
df_115.to_csv("../data/115_laws_text.csv", index = False)

# 114th

In [18]:
# import data
laws_114 = pd.read_csv("../data/114_laws.csv")
df_114 = pd.DataFrame(columns = ['Legislation Number', 'URL', 'Congress', 'Title', 'Sponsor',
        'Date of Introduction', "Chamber", "Bill Number", "Text"])

# create columns
laws_114["Chamber"] = laws_114["Legislation Number"].apply(get_chamber)
laws_114["Bill Number"] = laws_114["Legislation Number"].apply(get_bill_number)

# run function to get text
for index, row in laws_114.iterrows():
    congress = 114
    chamber = row["Chamber"]
    bill_no = row["Bill Number"]
    legislation_number = row["Legislation Number"]
    url = row["URL"]
    title = row["Title"]
    sponsor = row["Sponsor"]
    date_intro = row["Date of Introduction"]
    text = get_bill_text(congress, chamber, bill_no, api_key)
    new_row = {'Legislation Number': legislation_number, 
                    'URL': url, 
                    'Congress': congress, 
                    'Title': title, 
                    'Sponsor': sponsor,
                    'Date of Introduction': date_intro,
                    "Chamber": chamber,
                    "Bill Number": bill_no,
                    "Text": text}
    df_114.loc[len(df_114)] = new_row

/Users/natehiatt/miniconda3/envs/capstone/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
/Users/natehiatt/miniconda3/envs/capstone/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keywo

In [29]:
df_114.info()

<class 'pandas.core.frame.DataFrame'>
Index: 329 entries, 0 to 328
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Legislation Number    329 non-null    object
 1   URL                   329 non-null    object
 2   Congress              329 non-null    int64 
 3   Title                 329 non-null    object
 4   Sponsor               329 non-null    object
 5   Date of Introduction  329 non-null    object
 6   Chamber               329 non-null    object
 7   Bill Number           329 non-null    object
 8   Text                  329 non-null    object
dtypes: int64(1), object(8)
memory usage: 25.7+ KB


In [20]:
df_114.to_csv("../data/114_laws_text.csv", index = False)

# Making into function

In [31]:
# funciton
def collect_text(congress_no, laws_dataframe, text_dataframe, api_key):
    laws_dataframe["Chamber"] = laws_dataframe["Legislation Number"].apply(get_chamber)
    laws_dataframe["Bill Number"] = laws_dataframe["Legislation Number"].apply(get_bill_number)
    text_dataframe = pd.DataFrame(columns = ['Legislation Number', 'URL', 'Congress', 'Title', 'Sponsor',
        'Date of Introduction', "Chamber", "Bill Number", "Text"])
    for index, row in laws_dataframe.iterrows():
        congress = congress_no
        chamber = row["Chamber"]
        bill_no = row["Bill Number"]
        legislation_number = row["Legislation Number"]
        url = row["URL"]
        title = row["Title"]
        sponsor = row["Sponsor"]
        date_intro = row["Date of Introduction"]
        text = get_bill_text(congress, chamber, bill_no, api_key)
        new_row = {'Legislation Number': legislation_number, 
                    'URL': url, 
                    'Congress': congress, 
                    'Title': title, 
                    'Sponsor': sponsor,
                    'Date of Introduction': date_intro,
                    "Chamber": chamber,
                    "Bill Number": bill_no,
                    "Text": text}
        text_dataframe.loc[len(text_dataframe)] = new_row
    return text_dataframe

# 113th

In [27]:
laws_113 = pd.read_csv("../data/113_laws.csv")
df_113 = pd.DataFrame(columns = ['Legislation Number', 'URL', 'Congress', 'Title', 'Sponsor',
        'Date of Introduction', "Chamber", "Bill Number", "Text"])

collect_text(113, laws_113, df_113, api_key)

/Users/natehiatt/miniconda3/envs/capstone/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
/Users/natehiatt/miniconda3/envs/capstone/lib/python3.11/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keywo

,Legislation Number,URL,Congress,Title,Sponsor,Date of Introduction,Chamber,Bill Number,Text
0,H.R. 5859,https://www.congress.gov/bill/113th-congress/h...,113,Ukraine Freedom Support Act of 2014,"Gerlach, Jim [Rep.-R-PA-6]",12/11/2014,hr,5859,This Act may be cited as the “Ukraine Freedom ...
1,H.R. 5816,https://www.congress.gov/bill/113th-congress/h...,113,To extend the authorization for the United Sta...,"Royce, Edward R. [Rep.-R-CA-39]",12/09/2014,hr,5816,"in section 207(a) (22 U.S.C. 6435(a)), by stri..."
2,H.R. 5771,https://www.congress.gov/bill/113th-congress/h...,113,Tax Increase Prevention Act of 2014,"Camp, Dave [Rep.-R-MI-4]",12/01/2014,hr,5771,This division may be cited as the “Tax Increas...
3,H.R. 5739,https://www.congress.gov/bill/113th-congress/h...,113,No Social Security for Nazis Act,"Johnson, Sam [Rep.-R-TX-3]",11/19/2014,hr,5739,This Act may be cited as the “No Social Securi...
4,H.R. 5728,https://www.congress.gov/bill/113th-congress/h...,113,STELA Reauthorization Act of 2014,"Upton, Fred [Rep.-R-MI-6]",11/18/2014,hr,5728,This Act may be cited as the “STELA Reauthoriz...
...,...,...,...,...,...,...,...,...,...
291,S. 23,https://www.congress.gov/bill/113th-congress/s...,113,Sleeping Bear Dunes National Lakeshore Conserv...,"Levin, Carl [Sen.-D-MI]",01/22/2013,s,23,This Act may be cited as the “Sleeping Bear Du...
292,S.J.Res. 40,https://www.congress.gov/bill/113th-congress/s...,113,A joint resolution providing for the appointme...,"Leahy, Patrick J. [Sen.-D-VT]",07/08/2014,sjres,40,"That, in accordance with section 5581 of the ..."
293,S.J.Res. 32,https://www.congress.gov/bill/113th-congress/s...,113,A joint resolution providing for the reappoint...,"Leahy, Patrick J. [Sen.-D-VT]",03/04/2014,sjres,32,"That, in accordance with section 5581 of the ..."
294,S.J.Res. 29,https://www.congress.gov/bill/113th-congress/s...,113,A joint resolution providing for the appointme...,"Leahy, Patrick J. [Sen.-D-VT]",12/11/2013,sjres,29,"That, in accordance with section 5581 of the ..."


In [28]:
df_113.to_csv("../data/113_laws_text.csv", index = False)

# 112

In [178]:
# rewriting this function since the previous one wouldnt work if the last
# entry didn't contain and xml link.

def get_bill_text(congress, chamber, bill_no, api_key):
    url = f"https://api.congress.gov/v3/bill/{congress}/{chamber}/{bill_no}/text?api_key={api_key}"
    response = requests.get(url)
    data = response.json()
    try:
        if response.status_code == 200:
            last_xml_url = None
            for entry in data['textVersions']:
                formats = entry['formats']
                for format_entry in formats:
                    if format_entry['type'] == 'Formatted XML':
                        text_url = format_entry['url']
            get_text = requests.get(text_url)
            soup = BeautifulSoup(get_text.content, 'xml')
            legis_body = soup.find('legis-body')
            
            text = legis_body.get_text()
            text = text.replace('\u2001', '')
            return text
        else:
            print(f"Failed to fetch bill data for Congress {congress}, Bill Number {bill_no}")
            return None
    except AttributeError:
        pass


In [186]:
congress = 112
chamber = "sj"
bill_no = 44

url = f"https://api.congress.gov/v3/bill/{congress}/{chamber}/{bill_no}/text?api_key={api_key}"
url

'https://api.congress.gov/v3/bill/112/sj/44/text?api_key=jDJYSv2qFcQrFUq7kQEZB3zo019P2QBrfotk4bs1'

In [179]:
laws_112 = pd.read_csv("../data/112_laws.csv")
df_112 = pd.DataFrame(columns = ['Legislation Number', 'URL', 'Congress', 'Title', 'Sponsor',
        'Date of Introduction', "Chamber", "Bill Number", "Text"])

collect_text(112, laws_112, df_112, api_key)

,Legislation Number,URL,Congress,Title,Sponsor,Date of Introduction,Chamber,Bill Number,Text
0,H.R. 6671,https://www.congress.gov/bill/112th-congress/h...,112,Video Privacy Protection Act Amendments Act of...,"Goodlatte, Bob [Rep.-R-VA-6]",12/17/2012,hr,6671,\n1.Short titleThis Act may be cited as the\n\...
1,H.R. 6655,https://www.congress.gov/bill/112th-congress/h...,112,Protect our Kids Act of 2012,"Doggett, Lloyd [Rep.-D-TX-25]",12/13/2012,hr,6655,\n1.CommissionThis Act may be cited as the Pro...
2,H.R. 6634,https://www.congress.gov/bill/112th-congress/h...,112,To change the effective date for the Internet ...,"Cantor, Eric [Rep.-R-VA-7]",12/05/2012,hr,6634,\n1.Changed effective date for financial discl...
3,H.R. 6621,https://www.congress.gov/bill/112th-congress/h...,112,To correct and improve certain provisions of t...,"Smith, Lamar [Rep.-R-TX-21]",11/30/2012,hr,6621,\n1.Technical\n\t\t\t corrections\n(a)Advice o...
4,H.R. 6620,https://www.congress.gov/bill/112th-congress/h...,112,Former Presidents Protection Act of 2012,"Gowdy, Trey [Rep.-R-SC-4]",11/30/2012,hr,6620,\n1.Short titleThis Act may be cited as the\n\...
...,...,...,...,...,...,...,...,...,...
279,S.J.Res. 44,https://www.congress.gov/bill/112th-congress/s...,112,A joint resolution granting the consent of Con...,"Kohl, Herb [Sen.-D-WI]",06/14/2012,sjres,44,None
280,S.J.Res. 22,https://www.congress.gov/bill/112th-congress/s...,112,A joint resolution to grant the consent of Con...,"McCaskill, Claire [Sen.-D-MO]",06/28/2011,sjres,22,None
281,S.J.Res. 9,https://www.congress.gov/bill/112th-congress/s...,112,A joint resolution providing for the reappoint...,"Leahy, Patrick J. [Sen.-D-VT]",02/28/2011,sjres,9,None
282,S.J.Res. 8,https://www.congress.gov/bill/112th-congress/s...,112,A joint resolution providing for the appointme...,"Leahy, Patrick J. [Sen.-D-VT]",02/28/2011,sjres,8,None


In [183]:
df_112

,Legislation Number,URL,Congress,Title,Sponsor,Date of Introduction,Chamber,Bill Number,Text
